In [ ]:
#pip install janome

In [1]:
import numpy as np
import pandas as pd
import janome.tokenizer as jt

# Japanese stopwords
stopwords = [
    'の', 'に', 'は', 'を', 'で', 'と', 'が', 'た', 'し', 'て', 'ない',
    'も', 'なる', 'という', 'ある', 'する', 'です', 'ます', 'ため', 'から',
    'こと', 'よう', 'など', 'それ', 'これ', 'あれ', 'どれ', 'ここ', 'そこ',
    'あそこ', 'どこ', 'だれ', '誰', '何', 'なぜ', 'どう', 'いつ', '今',
    '前', '後', '中', '上', '下', '間', '時', '日', '年', '月', '週間',
    '朝', '昼', '夜', '午前', '午後', '明日', '今日', '昨日', '時点', '場所',
    '人', '私', '彼', '彼女', '自分', '一つ', '二つ', '三つ', '四つ', '五つ',
    '六つ', '七つ', '八つ', '九つ', '十', '百', '千', '万', '一', '二',
    '三', '四', '五', '六', '七', '八', '九', '十', '百', '千', '万'
]

# Japanese punctuation
punctuation = [
    '、', '。', '！', '？', '・', '〜', '「', '」', '『', '』', '（', '）', '［', '］',
    '【', '】', '｛', '｝', '〈', '〉', '《', '》', '―', '…', '‥', '：', '；', '＃',
    '＠', '＄', '％', '＆', '＊', '＋', '－', '／', '＝', '＜＞', '＜', '＞', '＾', '￣', '｜'
]

def tokenize_japanese(text):
    tokenizer = jt.Tokenizer()
    tokens = [token.surface for token in tokenizer.tokenize(text)]
    return tokens

def text_clean(text):
    tokens = tokenize_japanese(text)
    clean_tokens = [token for token in tokens if token not in stopwords and token not in punctuation]
    return clean_tokens

# Positive and negative sentences
positive_sentences = [
    "日本語が面白いです。",
    "食べ物が美味しいです。",
    "自然の景色が素晴らしいです。",
    "友達と楽しい時間を過ごしました。",
    "笑顔が素敵です。"
]

negative_sentences = [
    "雨が嫌いです。",
    "疲れています。",
    "遅刻しました。",
    "失敗しました。",
    "悲しいニュースがありました。"
]

# Assign labels
positive_labels = np.ones(len(positive_sentences))
negative_labels = np.zeros(len(negative_sentences))

# Combine sentences


In [2]:
def build_freqs(sentences, labels):
    freqs = {}
    for label, sentence in zip(labels, sentences):
        clean_tokens = text_clean(sentence)
        for token in clean_tokens:
            pair = (token, label)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

# Positive and negative sentences
sentences = positive_sentences + negative_sentences
labels = np.concatenate((positive_labels, negative_labels))

# Calculate frequencies
freqs = build_freqs(sentences, labels)
print(freqs)


{('日本語', 1.0): 1, ('面白い', 1.0): 1, ('食べ物', 1.0): 1, ('美味しい', 1.0): 1, ('自然', 1.0): 1, ('景色', 1.0): 1, ('素晴らしい', 1.0): 1, ('友達', 1.0): 1, ('楽しい', 1.0): 1, ('時間', 1.0): 1, ('過ごし', 1.0): 1, ('まし', 1.0): 1, ('笑顔', 1.0): 1, ('素敵', 1.0): 1, ('雨', 0.0): 1, ('嫌い', 0.0): 1, ('疲れ', 0.0): 1, ('い', 0.0): 1, ('遅刻', 0.0): 1, ('まし', 0.0): 3, ('失敗', 0.0): 1, ('悲しい', 0.0): 1, ('ニュース', 0.0): 1, ('あり', 0.0): 1}


In [22]:
import math

def sentiment_score(sentence, freqs):
    score = 0
    tokens = text_clean(sentence)
    for token in tokens:
        if (token, 1) in freqs:
            score += math.log(freqs[(token, 1)] + 1)
        if (token, 0) in freqs:
            score -= math.log(freqs[(token, 0)] + 1)
    if score > 0.5:
        pred = 'positive'
    else:
        pred = 'negative'
    
    return score, pred

In [23]:
sentence1 = "これは面白い"
sentence2 = "これはよくない."
score1 = sentiment_score(sentence1, freqs)
score2 = sentiment_score(sentence2, freqs)
print(f"Sentence 1 score: {score1}")
print(f"Sentence 2 score: {score2}")


Sentence 1 score: (0.6931471805599453, 'positive')
Sentence 2 score: (0, 'negative')


In [24]:
import pandas as pd

positive_sentences = [
    "今日はとてもいい天気です。",
    "このレストランの料理は美味しいです。",
    "彼はとても親切な人です。",
    "この映画は面白かったです。",
    "彼女の笑顔が素敵です。",
    "この本は感動的なストーリーです。",
    "新しいアルバムが素晴らしいです。",
    "彼のパフォーマンスはすばらしかったです。",
    "友達との時間は楽しいです。",
    "この景色は素晴らしいです。"
]

negative_sentences = [
    "電車が遅れてイライラします。",
    "この商品は期待外れでした。",
    "彼の態度が嫌です。",
    "この映画はつまらなかったです。",
    "彼女の態度が冷たいです。",
    "この本はつまらない内容です。",
    "新しい商品の品質が低いです。",
    "彼の仕事ぶりに不満があります。",
    "ストレスがたまります。",
    "この場所は魅力がありません。"
]

data = {
    'sentence': positive_sentences + negative_sentences,
    'label': ['positive'] * 10 + ['negative'] * 10
}

df = pd.DataFrame(data)



In [25]:
df

,sentence,label
0,今日はとてもいい天気です。,positive
1,このレストランの料理は美味しいです。,positive
2,彼はとても親切な人です。,positive
3,この映画は面白かったです。,positive
4,彼女の笑顔が素敵です。,positive
5,この本は感動的なストーリーです。,positive
6,新しいアルバムが素晴らしいです。,positive
7,彼のパフォーマンスはすばらしかったです。,positive
8,友達との時間は楽しいです。,positive
9,この景色は素晴らしいです。,positive


In [26]:
# Define a function to apply sentiment_score and append score and prediction columns
def predict_sentiment(row, freqs):
    sentence = row['sentence']
    score, predicted_label = sentiment_score(sentence, freqs)
    row['score'] = score
    row['prediction'] = predicted_label
    return row

# Apply the predict_sentiment function to each row of the DataFrame
df = df.apply(lambda row: predict_sentiment(row, freqs), axis=1)

# Print the updated DataFrame with score and prediction columns
df


,sentence,label,score,prediction
0,今日はとてもいい天気です。,positive,0.000000,negative
1,このレストランの料理は美味しいです。,positive,0.693147,positive
2,彼はとても親切な人です。,positive,0.000000,negative
3,この映画は面白かったです。,positive,0.000000,negative
4,彼女の笑顔が素敵です。,positive,1.386294,positive
5,この本は感動的なストーリーです。,positive,0.000000,negative
6,新しいアルバムが素晴らしいです。,positive,0.693147,positive
7,彼のパフォーマンスはすばらしかったです。,positive,0.000000,negative
8,友達との時間は楽しいです。,positive,2.079442,positive
9,この景色は素晴らしいです。,positive,1.386294,positive


In [27]:
correct_predictions = df['label'] == df['prediction']
accuracy = correct_predictions.mean() * 100
print("Accuracy: {:.2f}%".format(accuracy))


Accuracy: 75.00%
